# Attribution Maps Workflow
This notebook demonstrates loading models, training them, printing model layers, and applying the Grad-CAM workflow.

In [1]:
# Import necessary modules
from models import PneumoniaCNN, PneumoniaResNet
from grad_cam import GradCAM
from train_cnn import train_net, eval_net
from dataloaders import train_loader, val_loader, test_loader
import torch
from torch import nn, optim

In [2]:
# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_model = PneumoniaCNN().to(device)
resnet_model = PneumoniaResNet().to(device)

c:\Users\paesc\anaconda3\envs\py311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\paesc\anaconda3\envs\py311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Train models
criterion = nn.CrossEntropyLoss()
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=0.001)
optimizer_resnet = optim.Adam(resnet_model.parameters(), lr=0.001)

print("Training CNN Model...")
train_net(cnn_model, train_loader, val_loader, criterion, optimizer_cnn, device, epochs=5)
eval_net(cnn_model, test_loader, device)



# print("Training ResNet Model...")
# train_net(resnet_model, train_loader, val_loader, criterion, optimizer_resnet, device, epochs=5)
# eval_net(resnet_model, test_loader, device)


# save the model
torch.save(cnn_model.state_dict(), 'model_state_dicts/cnn_model.pth')
torch.save(resnet_model.state_dict(), 'model_state_dicts/resnet_model.pth')


Training CNN Model...


Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
# Print model layers
print("CNN Model Layers:")
print(cnn_model)

# print("ResNet Model Layers:")
# print(resnet_model)

CNN Model Layers:
PneumoniaCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


In [ ]:
# Apply Grad-CAM workflow
target_layer = cnn_model.conv2
grad_cam = GradCAM(cnn_model, target_layer)

# Get a sample input
data_iter = iter(val_loader)
images, labels = next(data_iter)
input_tensor = images[0].unsqueeze(0).to(device)

# Generate CAM
cam = grad_cam.generate_cam(input_tensor)

# Visualize
import matplotlib.pyplot as plt
image = images[0].permute(1, 2, 0).numpy()
image = (image * 0.5 + 0.5) * 255  # Denormalize
image = image.astype(np.uint8)
overlay = grad_cam.overlay_cam(image, cam)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image, cmap="gray")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Grad-CAM")
plt.imshow(overlay)
plt.axis("off")
plt.show()

c:\Users\paesc\anaconda3\envs\py311\Lib\site-packages\torch\nn\modules\module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
